# Quantization 비교 실험 - EchoNet-Dynamic EF 회귀 모델

이 노트북은 Post-Training Quantization (PTQ)와 Quantization-Aware Training (QAT)를 비교하는 실험을 수행합니다.

## 실험 목표
- Baseline (FP32) 모델 학습
- PTQ 적용 및 성능 측정
- QAT 적용 및 성능 측정
- 세 모델의 MAE, Model Size, Latency 비교



## 1. 환경 체크 및 설정


In [ ]:
# GPU/CPU 체크
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
else:
    print("Using CPU")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\nUsing device: {device}")


In [ ]:
# 필요한 패키지 설치
%pip install -q opencv-python pandas tqdm matplotlib seaborn


In [ ]:
# 프로젝트 파일들을 Colab에 업로드하거나 GitHub에서 클론
# 방법 1: 파일 업로드 (Colab의 파일 메뉴에서)
# 방법 2: GitHub에서 클론
# !git clone <your-repo-url>
# %cd <repo-directory>

# 또는 직접 파일들을 복사/붙여넣기

# 현재 디렉토리 확인
import os
print(f"Current directory: {os.getcwd()}")
print(f"Files in current directory: {os.listdir('.')}")


## 2. 데이터 준비


In [ ]:
# TODO: 샘플 EchoNet 데이터를 /content/sample_echonet/ 에 업로드하세요
# 구조:
# /content/sample_echonet/
#   ├── Videos/
#   │   ├── video1.avi
#   │   ├── video2.avi
#   │   └── ...
#   └── FileList.csv  # FileName, EF 컬럼 포함

# 데이터 경로 확인
from pathlib import Path

BASE_DIR = Path("/content/sample_echonet")  # TODO: 실제 경로로 수정
VIDEO_DIR = BASE_DIR / "Videos"
FILELIST_PATH = BASE_DIR / "FileList.csv"

print(f"Video directory exists: {VIDEO_DIR.exists()}")
print(f"FileList exists: {FILELIST_PATH.exists()}")

if FILELIST_PATH.exists():
    import pandas as pd
    df = pd.read_csv(FILELIST_PATH)
    print(f"\nFileList shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"\nFirst few rows:")
    print(df.head())


## 3. 모듈 import 및 설정


In [ ]:
# 프로젝트 모듈 import
# 노트북이 프로젝트 루트에 있다고 가정
import sys
sys.path.append('..')  # 상위 디렉토리 추가

# 또는 절대 경로 사용
# sys.path.append('/content/your-project-path')

import config
from dataset import get_dataloaders
from model import create_model
from train import train_model, load_checkpoint
from quant_utils import apply_ptq, apply_qat
from metrics import evaluate_model_performance
from main_ptq import run_ptq_experiment
from main_qat import run_qat_experiment
from run_all import run_all_experiments
import torch.nn as nn

# Config 업데이트 (Colab 경로에 맞게)
config.BASE_DIR = BASE_DIR
config.VIDEO_DIR = VIDEO_DIR
config.FILELIST_PATH = FILELIST_PATH
config.DEVICE = device

print("Modules imported successfully!")
print(f"Config updated:")
print(f"  BASE_DIR: {config.BASE_DIR}")
print(f"  VIDEO_DIR: {config.VIDEO_DIR}")
print(f"  FILELIST_PATH: {config.FILELIST_PATH}")


## 4. 데이터 로더 생성


In [ ]:
# Train/Val 데이터 로더 생성
train_loader, val_loader = get_dataloaders(
    video_dir=config.VIDEO_DIR,
    filelist_path=config.FILELIST_PATH,
    batch_size=config.BATCH_SIZE,
    num_workers=0,  # Colab에서는 0 권장
    num_frames=config.NUM_FRAMES,
    img_size=config.IMG_SIZE,
)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")

# 샘플 데이터 확인
sample_video, sample_ef = next(iter(train_loader))
print(f"\nSample video shape: {sample_video.shape}")  # (B, N, C, H, W)
print(f"Sample EF label: {sample_ef[:5]}")  # 첫 5개 샘플


## 5. Baseline 모델 학습


In [ ]:
# Baseline 모델 생성
model = create_model()
print(f"Model created: {model.__class__.__name__}")

# 모델 파라미터 수 확인
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")


In [ ]:
# Baseline 모델 학습
# TODO: 이미 학습된 모델이 있다면 이 셀을 스킵하고 다음 셀에서 로드하세요

history = train_model(
    model,
    train_loader,
    val_loader,
    num_epochs=config.NUM_EPOCHS,
    learning_rate=config.LEARNING_RATE,
    weight_decay=config.WEIGHT_DECAY,
    device=config.DEVICE,
    checkpoint_dir=config.CHECKPOINT_DIR,
    save_best=True,
    verbose=True,
)


In [ ]:
# 학습 히스토리 시각화 (optional)
import matplotlib.pyplot as plt

if 'history' in locals():
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # Loss
    axes[0].plot(history['train_loss'], label='Train Loss')
    axes[0].plot(history['val_loss'], label='Val Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training and Validation Loss')
    axes[0].legend()
    axes[0].grid(True)
    
    # MAE
    axes[1].plot(history['val_mae'], label='Val MAE')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('MAE')
    axes[1].set_title('Validation MAE')
    axes[1].legend()
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.show()


## 6. Baseline 모델 성능 측정


In [ ]:
# Baseline 모델 로드 (이미 학습된 경우)
baseline_checkpoint = config.CHECKPOINT_DIR / "best_model.pth"

if baseline_checkpoint.exists():
    model = create_model()
    checkpoint = load_checkpoint(model, baseline_checkpoint, config.DEVICE)
    print(f"Loaded baseline model from {baseline_checkpoint}")
    if 'val_mae' in checkpoint:
        print(f"Best MAE: {checkpoint['val_mae']:.4f}")
else:
    print("Baseline checkpoint not found. Please train the model first.")

# Baseline 성능 측정
criterion = nn.MSELoss()
baseline_performance = evaluate_model_performance(
    model,
    val_loader,
    criterion,
    device=config.DEVICE,
    verbose=True,
)

print("\n=== Baseline (FP32) Performance ===")
print(f"MAE: {baseline_performance['mae']:.4f}")
print(f"Model Size: {baseline_performance['size_mb']:.4f} MB")
print(f"Latency: {baseline_performance['latency_ms']:.4f} ms/video")


## 7. Post-Training Quantization (PTQ)


In [ ]:
# PTQ 실험 실행
ptq_results = run_ptq_experiment(
    baseline_checkpoint_path=baseline_checkpoint,
    save_results=True,
    verbose=True,
)


## 8. Quantization-Aware Training (QAT)


In [ ]:
# QAT 실험 실행
qat_results = run_qat_experiment(
    baseline_checkpoint_path=baseline_checkpoint,
    save_results=True,
    verbose=True,
)


## 9. 최종 비교 결과


In [ ]:
# 전체 결과 비교 테이블 생성
import pandas as pd

comparison_data = [
    {
        'Model': 'FP32 Baseline',
        'Precision': 'FP32',
        'Size (MB)': f"{baseline_performance['size_mb']:.4f}",
        'MAE': f"{baseline_performance['mae']:.4f}",
        'Latency (ms)': f"{baseline_performance['latency_ms']:.4f}",
    },
    {
        'Model': 'PTQ',
        'Precision': 'INT8',
        'Size (MB)': f"{ptq_results['ptq']['size_mb']:.4f}",
        'MAE': f"{ptq_results['ptq']['mae']:.4f}",
        'Latency (ms)': f"{ptq_results['ptq']['latency_ms']:.4f}",
    },
    {
        'Model': 'QAT',
        'Precision': 'INT8',
        'Size (MB)': f"{qat_results['qat']['size_mb']:.4f}",
        'MAE': f"{qat_results['qat']['mae']:.4f}",
        'Latency (ms)': f"{qat_results['qat']['latency_ms']:.4f}",
    },
]

comparison_df = pd.DataFrame(comparison_data)
print("="*60)
print("Final Comparison Results")
print("="*60)
print(comparison_df.to_string(index=False))


In [ ]:
# 상세 비교 분석
print("\n" + "="*60)
print("Detailed Comparison Analysis")
print("="*60)

print(f"\n1. Accuracy (MAE):")
print(f"   Baseline: {baseline_performance['mae']:.4f}")
print(f"   PTQ:      {ptq_results['ptq']['mae']:.4f} (Drop: {ptq_results['ptq']['mae_drop']:.4f}, {ptq_results['ptq']['mae_drop_percent']:.2f}%)")
print(f"   QAT:      {qat_results['qat']['mae']:.4f} (Drop: {qat_results['qat']['mae_drop']:.4f}, {qat_results['qat']['mae_drop_percent']:.2f}%)")

mae_diff = ptq_results['ptq']['mae'] - qat_results['qat']['mae']
print(f"   QAT vs PTQ: {mae_diff:.4f} {'(QAT better)' if mae_diff > 0 else '(PTQ better)'}")

print(f"\n2. Model Size:")
print(f"   Baseline: {baseline_performance['size_mb']:.4f} MB")
print(f"   PTQ:      {ptq_results['ptq']['size_mb']:.4f} MB (Reduction: {ptq_results['ptq']['size_reduction_mb']:.4f} MB, {ptq_results['ptq']['size_reduction_percent']:.2f}%)")
print(f"   QAT:      {qat_results['qat']['size_mb']:.4f} MB (Reduction: {qat_results['qat']['size_reduction_mb']:.4f} MB, {qat_results['qat']['size_reduction_percent']:.2f}%)")

print(f"\n3. Latency:")
print(f"   Baseline: {baseline_performance['latency_ms']:.4f} ms/video")
print(f"   PTQ:      {ptq_results['ptq']['latency_ms']:.4f} ms/video (Improvement: {ptq_results['ptq']['latency_improvement_ms']:.4f} ms, {ptq_results['ptq']['latency_improvement_percent']:.2f}%)")
print(f"   QAT:      {qat_results['qat']['latency_ms']:.4f} ms/video (Improvement: {qat_results['qat']['latency_improvement_ms']:.4f} ms, {qat_results['qat']['latency_improvement_percent']:.2f}%)")


## 10. 결과 시각화 (Optional)


In [ ]:
# 결과 시각화
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

models = ['Baseline\n(FP32)', 'PTQ\n(INT8)', 'QAT\n(INT8)']
x = np.arange(len(models))

# MAE 비교
mae_values = [
    baseline_performance['mae'],
    ptq_results['ptq']['mae'],
    qat_results['qat']['mae'],
]
axes[0].bar(x, mae_values, color=['blue', 'orange', 'green'])
axes[0].set_ylabel('MAE')
axes[0].set_title('Mean Absolute Error')
axes[0].set_xticks(x)
axes[0].set_xticklabels(models)
axes[0].grid(True, axis='y', alpha=0.3)

# Model Size 비교
size_values = [
    baseline_performance['size_mb'],
    ptq_results['ptq']['size_mb'],
    qat_results['qat']['size_mb'],
]
axes[1].bar(x, size_values, color=['blue', 'orange', 'green'])
axes[1].set_ylabel('Size (MB)')
axes[1].set_title('Model Size')
axes[1].set_xticks(x)
axes[1].set_xticklabels(models)
axes[1].grid(True, axis='y', alpha=0.3)

# Latency 비교
latency_values = [
    baseline_performance['latency_ms'],
    ptq_results['ptq']['latency_ms'],
    qat_results['qat']['latency_ms'],
]
axes[2].bar(x, latency_values, color=['blue', 'orange', 'green'])
axes[2].set_ylabel('Latency (ms/video)')
axes[2].set_title('Inference Latency')
axes[2].set_xticks(x)
axes[2].set_xticklabels(models)
axes[2].grid(True, axis='y', alpha=0.3)

plt.tight_layout()
plt.show()


## 11. 결과 파일 다운로드 (Colab)


In [ ]:
# 결과 파일 다운로드
from google.colab import files
import zipfile
from pathlib import Path

# 결과 디렉토리 확인
results_dir = config.RESULTS_DIR
print(f"Results directory: {results_dir}")
print(f"Files in results directory:")
for f in results_dir.glob("*"):
    print(f"  - {f.name}")

# 결과 파일들을 zip으로 압축
zip_path = "/content/quantization_results.zip"
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for file_path in results_dir.glob("*"):
        zipf.write(file_path, file_path.name)
    # 체크포인트도 포함 (선택사항)
    # for file_path in config.CHECKPOINT_DIR.glob("*.pth"):
    #     zipf.write(file_path, f"checkpoints/{file_path.name}")

print(f"\nResults zipped to: {zip_path}")

# 다운로드
files.download(zip_path)


## 12. 전체 실험 한 번에 실행 (Alternative)

위의 단계별 실행 대신, `run_all.py`를 사용하여 전체 실험을 한 번에 실행할 수도 있습니다.


In [ ]:
# 전체 실험 한 번에 실행 (선택사항)
# 주의: 이 방법은 baseline을 새로 학습합니다
# 이미 학습된 모델이 있다면 위의 단계별 실행을 권장합니다

# all_results = run_all_experiments(
#     train_baseline=True,  # False로 설정하면 기존 체크포인트 사용
#     save_results=True,
#     verbose=True,
# )
